In [ ]:
import os
import pyvista as pv

pv.set_jupyter_backend('static')
os.environ['DISPLAY'] = ':99.0'
os.environ['PYVISTA_OFF_SCREEM'] = 'true'

In [ ]:
stride = 10
mesh_path = '/mnt/d/knpob/4-data/20211229-DynaBreast4D/3dmd/6kmh_26marker_2/meshes'
marker_3dmd_path = 'output/marker_3dmd.pkl'
marker_vicon_path = '/mnt/d/knpob/4-data/20211229-DynaBreast4D/vicon/6kmh_26marker_2.csv'
export_folder = "output"

# Load data

## 3dMD

In [ ]:
import numpy as np
from mesh4d import utils

fps_3dmd = round(120 / stride)
marker_3dmd = utils.load_pkl_object(marker_3dmd_path)
total_3dmd = len(marker_3dmd)

In [ ]:
from mesh4d import obj3d

mesh_ls, texture_ls = obj3d.load_mesh_series(mesh_path, start=0, end=999, stride=stride)

## Vicon

In [ ]:
def nan_to_neighbor_mean(arr):
    arr_fill = np.copy(arr)
    idxs = np.where(np.isnan(arr_fill))

    for row, col in zip(*idxs):
        if row == 0:
            arr_fill[row, col] = arr_fill[row+1, col]

        elif row == len(arr_fill) - 1:
            arr_fill[row, col] = arr_fill[row-1, col]

        else:
            arr_fill[row, col] = (arr_fill[row-1, col] + arr_fill[row+1, col]) / 2

    return arr_fill

In [ ]:
import pandas as pd

df = pd.read_csv(marker_vicon_path, skiprows=4).iloc[:, 2:]
df

In [ ]:
marker_vicon = nan_to_neighbor_mean(df.values).reshape(len(df), -1, 3)
marker_vicon.shape

# Alignment distance

In [ ]:
from scipy.spatial import KDTree

def vicon_3dmd_dist(points_vicon, points_3dmd):
    tree = KDTree(points_3dmd)
    d, _ = tree.query(points_vicon)
    
    return d.mean()

## Before alignment

In [ ]:
from mesh4d import kps

vicon = kps.MarkerSet()
vicon.load_from_array(marker_vicon, start_time=0.0, fps=100, trans_cab=None)
vicon.interp_field()

In [ ]:
scene = pv.Plotter()
scene.open_movie(os.path.join(export_folder, 'diff_3dmd_vicon_raw.mp4'), framerate=round(120/stride))

dist_ls_raw = []

for frame, points_3dmd in enumerate(marker_3dmd):
    try:
        timestamp = frame / fps_3dmd
        points_vicon = vicon.get_time_coord(timestamp).get_points_coord()
        dist_ls_raw.append(vicon_3dmd_dist(points_vicon, points_3dmd))

        scene.clear()
        scene.add_points(points_vicon, point_size=10, color='green', render_points_as_spheres=True)
        scene.add_points(points_3dmd, point_size=5, color='goldenrod')
        scene.add_mesh(mesh_ls[frame], texture=texture_ls[frame], opacity=0.7)
        scene.camera_position = 'xy'
        scene.write_frame()
        
    except:
        pass

    # print progress
    percent = (frame + 1) / total_3dmd
    utils.progress_bar(percent, back_str=" exported the {}-th frame".format(frame))

scene.close()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(dist_ls_raw, label='raw')
plt.legend()
plt.xlabel('frame id')
plt.ylabel('dist (mm)')
f"std {np.std(dist_ls_raw)} mean {np.mean(dist_ls_raw)} min {np.min(dist_ls_raw)} max {np.max(dist_ls_raw)} (mm)"

## After spatial alignment

In [ ]:
vicon = kps.MarkerSet()
trans_cab = utils.load_pkl_object('output/vicon>>3dmd.pkl')
vicon.load_from_array(marker_vicon, start_time=0.0, fps=100, trans_cab=trans_cab)
vicon.interp_field()

In [ ]:
scene = pv.Plotter()
scene.open_movie(os.path.join(export_folder, 'diff_3dmd_vicon_space.mp4'), framerate=round(120/stride))

dist_ls_space = []

for frame, points_3dmd in enumerate(marker_3dmd):
    try:
        timestamp = frame / fps_3dmd
        points_vicon = vicon.get_time_coord(timestamp).get_points_coord()
        dist_ls_space.append(vicon_3dmd_dist(points_vicon, points_3dmd))

        scene.clear()
        scene.add_points(points_vicon, point_size=10, color='green', render_points_as_spheres=True)
        scene.add_points(points_3dmd, point_size=5, color='goldenrod')
        scene.add_mesh(mesh_ls[frame], texture=texture_ls[frame], opacity=0.7)
        scene.camera_position = 'xy'
        scene.write_frame()
        
    except:
        pass

    # print progress
    percent = (frame + 1) / total_3dmd
    utils.progress_bar(percent, back_str=" exported the {}-th frame".format(frame))

scene.close()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(dist_ls_raw, label='raw')
plt.plot(dist_ls_space, label='space aligned')
plt.legend()
plt.xlabel('frame id')
plt.ylabel('dist (mm)')
f"std {np.std(dist_ls_space)} mean {np.mean(dist_ls_space)} min {np.min(dist_ls_space)} max {np.max(dist_ls_space)} (mm)"

# Temporal alignment

## Round 1

In [ ]:
time_offset = np.linspace(-10, 10, 100)
dist_ls = []

for idx, offset in enumerate(time_offset):
    vicon = kps.MarkerSet()
    trans_cab = utils.load_pkl_object('output/vicon>>3dmd.pkl')
    vicon.load_from_array(marker_vicon, start_time=offset, fps=100, trans_cab=trans_cab)
    vicon.interp_field()

    dist_ls_time = []

    for frame, points_3dmd in enumerate(marker_3dmd):
        try:
            timestamp = frame / fps_3dmd
            points_vicon = vicon.get_time_coord(timestamp).get_points_coord()
            dist_ls_time.append(vicon_3dmd_dist(points_vicon, points_3dmd))
            
        except:
            pass
    
    if len(dist_ls_time) > 0:
        dist_ls.append(np.mean(dist_ls_time))

    else:
        dist_ls.append(np.nan)

    # print progress
    percent = (idx + 1) / len(time_offset)
    utils.progress_bar(percent, back_str=f" benchmarked time offset {offset}s")

In [ ]:
plt.plot(time_offset, dist_ls, label='dist')
plt.legend()
plt.xlabel('offset (s)')
plt.ylabel('dist (mm)')

idx_min = np.nanargmin(dist_ls)
offset_min = time_offset[idx_min]
f"optimal vicon time offset: {offset_min:.6f}s with dist {dist_ls[idx_min]} (mm)"

## Round 2

In [ ]:
time_offset = np.linspace(offset_min - 0.5, offset_min + 0.5, 100)
dist_ls = []

for idx, offset in enumerate(time_offset):
    vicon = kps.MarkerSet()
    trans_cab = utils.load_pkl_object('output/vicon>>3dmd.pkl')
    vicon.load_from_array(marker_vicon, start_time=offset, fps=100, trans_cab=trans_cab)
    vicon.interp_field()

    dist_ls_time = []

    for frame, points_3dmd in enumerate(marker_3dmd):
        try:
            timestamp = frame / fps_3dmd
            points_vicon = vicon.get_time_coord(timestamp).get_points_coord()
            dist_ls_time.append(vicon_3dmd_dist(points_vicon, points_3dmd))
            
        except:
            pass
    
    if len(dist_ls_time) > 0:
        dist_ls.append(np.mean(dist_ls_time))

    else:
        dist_ls.append(np.nan)

    # print progress
    percent = (idx + 1) / len(time_offset)
    utils.progress_bar(percent, back_str=f" benchmarked time offset {offset}s")

In [ ]:
plt.plot(time_offset, dist_ls, label='dist')
plt.legend()
plt.xlabel('offset (s)')
plt.ylabel('dist (mm)')

idx_min = np.nanargmin(dist_ls)
offset_min = time_offset[idx_min]
f"optimal vicon time offset: {offset_min:.6f}s with dist {dist_ls[idx_min]} (mm)"

In [ ]:
utils.save_pkl_object(offset_min, export_folder='output', export_name='vicon_start')

## Verification

In [ ]:
vicon = kps.MarkerSet()
trans_cab = utils.load_pkl_object('output/vicon>>3dmd.pkl')
vicon.load_from_array(marker_vicon, start_time=offset_min, fps=100, trans_cab=trans_cab)
vicon.interp_field()

In [ ]:
scene = pv.Plotter()
scene.open_movie(os.path.join(export_folder, 'diff_3dmd_vicon_time.mp4'), framerate=round(120/stride))

dist_ls_time = []

for frame, points_3dmd in enumerate(marker_3dmd):
    try:
        timestamp = frame / fps_3dmd
        points_vicon = vicon.get_time_coord(timestamp).get_points_coord()
        dist_ls_time.append(vicon_3dmd_dist(points_vicon, points_3dmd))

        scene.clear()
        scene.add_points(points_vicon, point_size=10, color='green', render_points_as_spheres=True)
        scene.add_points(points_3dmd, point_size=5, color='goldenrod')
        scene.add_mesh(mesh_ls[frame], texture=texture_ls[frame], opacity=0.7)
        scene.camera_position = 'xy'
        scene.write_frame()
        
    except:
        pass

    # print progress
    percent = (frame + 1) / total_3dmd
    utils.progress_bar(percent, back_str=" exported the {}-th frame".format(frame))

scene.close()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(dist_ls_raw, label='raw')
plt.plot(dist_ls_space, label='space aligned')
plt.plot(dist_ls_time, label='time aligned')
plt.legend()
plt.xlabel('frame id')
plt.ylabel('dist (mm)')
f"std {np.std(dist_ls_time)} mean {np.mean(dist_ls_time)} min {np.min(dist_ls_time)} max {np.max(dist_ls_time)} (mm)"